<a href="https://colab.research.google.com/github/aborundiya/NeuralNetwork/blob/master/ComparisonOfAllGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors
from matplotlib import animation, rc
from IPython.display import HTML
import numpy as np

In [0]:
class SN:

  def __init__(self, w_init, b_init, algo ):
    self.w = w_init
    self.b = b_init
    self.w_h = []
    self.b_h = []
    self.e_h = []
    self.algo = algo
  
  def sigmoid(self, w = None, b = None ):
    if w is None:
      w = self.w
    if b is None:
      b = self.b
    return 1.0 / ( 1.0 + np.exp(-(w * x + b )))
  
  def error(self, X, Y)